# Data Processing
Data cleaning and feature engineering

In [291]:
import pandas as pd
import numpy as np
import sklearn as sk
from datetime import date
import ast

## Read the dataset

In [292]:
original_df = pd.read_csv('test.csv', parse_dates=['host_since', 'first_review', 'last_review'])
print(original_df.columns)

Index(['id', 'name', 'description', 'property_type', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'latitude', 'longitude', 'host_since',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'room_type',
       'accommodates', 'bathrooms', 'bathrooms_text', 'bedrooms', 'beds',
       'amenities', 'has_availability', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'instant_bookable',
       'minimum_nights', 'maximum_nights', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review',
       'last_review', 'review_scores

In [293]:
original_df['bathrooms_text']

#  Calculate the proportion of True values
# proportion_true = original_df['host_identity_verified'].mean()

#  Print the result
# print(f"The proportion of True values is: {proportion_true:.2%}")


0            1.5 baths
1       3 shared baths
2               1 bath
3              3 baths
4               1 bath
             ...      
6722    1 private bath
6723     1 shared bath
6724    3 shared baths
6725            1 bath
6726    1 private bath
Name: bathrooms_text, Length: 6727, dtype: object

## Remove Unnecessary and Redundant Features
All text-based descriptions and reviews will not be parsed.
Neighborhood data is not necessary as this algorithm will utilize longitude and latitude for location data. 
host_idenity_verified will serve as the verification feature rather than parsing additional verifications
One feature for the number of host listings is plenty sufficient. 


In [294]:
df = original_df.drop(
    ["name", "description", "neighbourhood_cleansed", 
     "neighbourhood_group_cleansed", "host_verifications", 
     "calculated_host_listings_count_entire_homes", 
     "calculated_host_listings_count_private_rooms", 
     "calculated_host_listings_count_shared_rooms", 
     "reviews"], 
    axis=1
)

## Resolve Two Bathroom Columns
The bathrooms text is used for more recent scrapes and typically has the word "baths" following a float value for the number of bathrooms at the property. 

In [295]:

# Extract the numerical part from 'bathrooms_text'
df['extracted_bathrooms'] = (
    df['bathrooms_text']
    .str.extract(r'([\d\.]+)')  # Regex to capture numbers (including decimals)
    .astype(float)              # Convert to float
)

# Replace 'bathrooms' with the extracted value if it exists
df['bathrooms'] = df['extracted_bathrooms'].combine_first(df['bathrooms'])

# Drop the helper column if not needed
df.drop(columns=['extracted_bathrooms'], inplace=True)
df.drop(columns=['bathrooms_text'], inplace=True)

df['bathrooms']


0       1.5
1       3.0
2       1.0
3       3.0
4       1.0
       ... 
6722    1.0
6723    1.0
6724    3.0
6725    1.0
6726    1.0
Name: bathrooms, Length: 6727, dtype: float64

## Change Response Time to a Numerical Scale

In [296]:
# Find unique values
response_times = df['host_response_time'].dropna().unique().tolist()
print("Response times:", response_times)

# 1 = 'within an hour', 2 = 'within a day', 3 = 'within a few hours', 4 = 'a few days or more'
# Map string values to corresponding numerical values
response_time_mapping = {
    'within an hour': 1,
    'within a day': 2,
    'within a few hours': 3,
    'a few days or more': 4
}

# Replace the string values with numbers, leaving NaN values as is
df['host_response_time'] = df['host_response_time'].replace(response_time_mapping)

df

Response times: ['within an hour', 'within a day', 'within a few hours', 'a few days or more']


,id,property_type,latitude,longitude,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,...,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,3917,Entire home,40.744620,-73.904520,2013-05-07,NaN,NaN,NaN,False,1.0,...,2022-10-17,2023-09-17,5.00,5.00,4.96,5.00,5.00,4.96,4.96,1.22
1,1885,Private room in rental unit,40.753407,-73.934995,2012-08-11,1.0,99.0,23.0,False,727.0,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1305,Entire loft,40.677090,-73.943810,2012-05-14,NaN,NaN,NaN,False,1.0,...,2021-10-17,2022-10-31,4.94,5.00,4.90,4.90,4.97,4.68,4.90,0.88
3,19328,Entire rental unit,40.795760,-73.971570,2021-04-05,2.0,70.0,37.0,False,36.0,...,2023-05-29,2024-08-21,4.75,4.50,4.75,5.00,4.50,5.00,4.50,0.26
4,16511,Entire rental unit,40.713590,-73.955400,2014-10-07,3.0,100.0,75.0,False,1.0,...,2015-02-22,2024-08-31,4.82,4.89,4.69,4.97,4.96,4.94,4.73,1.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6722,7205,Private room in rental unit,40.637960,-73.951360,2015-09-12,1.0,100.0,100.0,False,2.0,...,2015-10-21,2023-11-18,4.71,4.75,4.64,4.88,4.90,4.65,4.71,3.01
6723,3954,Private room in rental unit,40.823720,-73.945460,2016-03-17,4.0,0.0,33.0,False,7.0,...,2018-08-31,2024-06-01,4.36,4.55,4.55,4.91,4.55,4.91,4.45,0.15
6724,1358,Private room in rental unit,40.755094,-73.937260,2012-08-11,1.0,99.0,23.0,False,727.0,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6725,2793,Entire rental unit,40.781580,-73.984780,2012-01-12,1.0,100.0,100.0,True,1.0,...,2021-12-01,2024-07-02,4.88,4.94,4.73,4.94,5.00,4.85,4.76,0.98


## Turn Binary Columns to 1s/0s

In [297]:
columns_to_convert = ['host_is_superhost', 'has_availability']
df[columns_to_convert] = df[columns_to_convert].replace({'True': 1, 'False': 0})

# Fill NaN values with 0
df[columns_to_convert] = df[columns_to_convert].fillna(0).astype(int)

## Resolve NaT and NaN Occurrences
As a naive approach, NaN numerical features are replaced with the means for each feature.
NaT dates are replaced with today's date. 
Missing binary values are replaced with false. 

In [298]:
# Find the columns with Na values 
na_columns = df.columns[df.isna().any()].tolist()
print(f"Columns with NA Value: {na_columns}")

# Replace missing numerical features with averages
for col in df.select_dtypes(include=['number']).columns:
    if col in na_columns:
        df[col].fillna(df[col].mean(), inplace=True)

# Replace missing dates with today's date (date only, not time)
for col in ['host_since', 'first_review', 'last_review']:
    if col in na_columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')  # Ensure proper datetime parsing
        df[col].fillna(pd.Timestamp(date.today()), inplace=True)

df['host_since'] = pd.to_datetime(df['host_since'])
df['first_review'] = pd.to_datetime(df['first_review'])
df['last_review'] = pd.to_datetime(df['last_review'])

df


Columns with NA Value: ['host_response_time', 'host_response_rate', 'host_acceptance_rate', 'bathrooms', 'bedrooms', 'beds', 'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']


,id,property_type,latitude,longitude,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,...,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,3917,Entire home,40.744620,-73.904520,2013-05-07,1.701004,91.509003,78.592453,0,1.0,...,2022-10-17,2023-09-17,5.000000,5.000000,4.960000,5.000000,5.000000,4.960000,4.960000,1.220000
1,1885,Private room in rental unit,40.753407,-73.934995,2012-08-11,1.000000,99.000000,23.000000,0,727.0,...,2024-11-20,2024-11-20,4.711449,4.748402,4.673496,4.813213,4.800491,4.710323,4.604796,1.234094
2,1305,Entire loft,40.677090,-73.943810,2012-05-14,1.701004,91.509003,78.592453,0,1.0,...,2021-10-17,2022-10-31,4.940000,5.000000,4.900000,4.900000,4.970000,4.680000,4.900000,0.880000
3,19328,Entire rental unit,40.795760,-73.971570,2021-04-05,2.000000,70.000000,37.000000,0,36.0,...,2023-05-29,2024-08-21,4.750000,4.500000,4.750000,5.000000,4.500000,5.000000,4.500000,0.260000
4,16511,Entire rental unit,40.713590,-73.955400,2014-10-07,3.000000,100.000000,75.000000,0,1.0,...,2015-02-22,2024-08-31,4.820000,4.890000,4.690000,4.970000,4.960000,4.940000,4.730000,1.350000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6722,7205,Private room in rental unit,40.637960,-73.951360,2015-09-12,1.000000,100.000000,100.000000,0,2.0,...,2015-10-21,2023-11-18,4.710000,4.750000,4.640000,4.880000,4.900000,4.650000,4.710000,3.010000
6723,3954,Private room in rental unit,40.823720,-73.945460,2016-03-17,4.000000,0.000000,33.000000,0,7.0,...,2018-08-31,2024-06-01,4.360000,4.550000,4.550000,4.910000,4.550000,4.910000,4.450000,0.150000
6724,1358,Private room in rental unit,40.755094,-73.937260,2012-08-11,1.000000,99.000000,23.000000,0,727.0,...,2024-11-20,2024-11-20,4.711449,4.748402,4.673496,4.813213,4.800491,4.710323,4.604796,1.234094
6725,2793,Entire rental unit,40.781580,-73.984780,2012-01-12,1.000000,100.000000,100.000000,1,1.0,...,2021-12-01,2024-07-02,4.880000,4.940000,4.730000,4.940000,5.000000,4.850000,4.760000,0.980000


In [299]:
# Check that all NA are gone
na_columns = df.columns[df.isna().any()].tolist()
print(f"Columns with NA Value: {na_columns}")

Columns with NA Value: []


## Convert DateTime Features to Numeric

In [300]:
# Convert datetime columns to numeric (e.g., number of days since the reference date)
df['host_since'] = (df['host_since'] - pd.Timestamp('1970-01-01')) // pd.Timedelta('1D')
df['first_review'] = (df['first_review'] - pd.Timestamp('1970-01-01')) // pd.Timedelta('1D')
df['last_review'] = (df['last_review'] - pd.Timestamp('1970-01-01')) // pd.Timedelta('1D')


## One-hot Encode room_type

In [301]:
# List all unique strings in 'property_type' and 'room_type' columns
property_types = df['property_type'].dropna().unique().tolist()
room_types = df['room_type'].dropna().unique().tolist()

print("Unique property types:", property_types)
print("Unique room types:", room_types)

# Upon investigation, property_type has too many categories and is largley redundant
df.drop(columns=['property_type'], inplace=True)

# one-hot encoding
df = pd.get_dummies(df, columns=['room_type'], prefix='room', dummy_na=False)

df

Unique property types: ['Entire home', 'Private room in rental unit', 'Entire loft', 'Entire rental unit', 'Room in hotel', 'Private room in home', 'Entire condo', 'Private room in townhouse', 'Camper/RV', 'Private room in guest suite', 'Room in boutique hotel', 'Entire townhouse', 'Shared room in condo', 'Entire guest suite', 'Shared room in rental unit', 'Entire vacation home', 'Entire serviced apartment', 'Private room in condo', 'Private room in serviced apartment', 'Private room in resort', 'Entire place', 'Room in serviced apartment', 'Tiny home', 'Private room in loft', 'Shared room in hostel', 'Shared room in home', 'Entire guesthouse', 'Entire bungalow', 'Private room in casa particular', 'Private room in guesthouse', 'Private room in bed and breakfast', 'Private room in vacation home', 'Private room in hostel', 'Private room', 'Room in aparthotel', 'Shared room in serviced apartment', 'Private room in villa', 'Houseboat', 'Shared room in casa particular', 'Private room in hou

,id,latitude,longitude,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,room_Entire home/apt,room_Hotel room,room_Private room,room_Shared room
0,3917,40.744620,-73.904520,15832,1.701004,91.509003,78.592453,0,1.0,12.0,...,4.960000,5.000000,5.000000,4.960000,4.960000,1.220000,1,0,0,0
1,1885,40.753407,-73.934995,15563,1.000000,99.000000,23.000000,0,727.0,1336.0,...,4.673496,4.813213,4.800491,4.710323,4.604796,1.234094,0,0,1,0
2,1305,40.677090,-73.943810,15474,1.701004,91.509003,78.592453,0,1.0,1.0,...,4.900000,4.900000,4.970000,4.680000,4.900000,0.880000,1,0,0,0
3,19328,40.795760,-73.971570,18722,2.000000,70.000000,37.000000,0,36.0,79.0,...,4.750000,5.000000,4.500000,5.000000,4.500000,0.260000,1,0,0,0
4,16511,40.713590,-73.955400,16350,3.000000,100.000000,75.000000,0,1.0,1.0,...,4.690000,4.970000,4.960000,4.940000,4.730000,1.350000,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6722,7205,40.637960,-73.951360,16690,1.000000,100.000000,100.000000,0,2.0,2.0,...,4.640000,4.880000,4.900000,4.650000,4.710000,3.010000,0,0,1,0
6723,3954,40.823720,-73.945460,16877,4.000000,0.000000,33.000000,0,7.0,8.0,...,4.550000,4.910000,4.550000,4.910000,4.450000,0.150000,0,0,1,0
6724,1358,40.755094,-73.937260,15563,1.000000,99.000000,23.000000,0,727.0,1336.0,...,4.673496,4.813213,4.800491,4.710323,4.604796,1.234094,0,0,1,0
6725,2793,40.781580,-73.984780,15351,1.000000,100.000000,100.000000,1,1.0,3.0,...,4.730000,4.940000,5.000000,4.850000,4.760000,0.980000,1,0,0,0


## One-hot Encode Key Amenities

In [302]:
# List of selected amenities to encode
selected_amenities = [
    "Air conditioning", "Kitchen", "Dedicated workspace", "Heating",
    "Hot water", "Refrigerator", "Free street parking", "Self check-in",
    "Shampoo", "Washer"
]

# Step 1: Ensure the amenities column is properly formatted as a list
df['amenities'] = df['amenities'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Step 2: Expand the list of amenities into individual rows
df_expanded = df.explode('amenities').reset_index()

# Check the columns after explode to avoid mismatch
print(df_expanded.columns)

# Rename columns for clarity
df_expanded = df_expanded[['index', 'amenities']]
df_expanded.columns = ['property_index', 'amenity']

# Step 3: Handle potential inconsistencies in strings
df_expanded['amenity'] = df_expanded['amenity'].str.strip()

# Step 4: Create one-hot encoding for the expanded amenities
df_expanded_one_hot = pd.get_dummies(df_expanded['amenity'], prefix='', prefix_sep='')

# Step 5: Group by property_index and aggregate by taking the max
df_expanded_one_hot = df_expanded_one_hot.groupby(df_expanded['property_index']).max()

# Step 6: Ensure selected amenities are in the columns, add them if not present
for amenity in selected_amenities:
    if amenity not in df_expanded_one_hot.columns:
        df_expanded_one_hot[amenity] = 0

# Align the columns with the selected amenities order
df_expanded_one_hot = df_expanded_one_hot[selected_amenities]

# Step 7: Merge the one-hot encoded amenities into the original df
df = df.merge(df_expanded_one_hot, left_index=True, right_index=True, how='left')

# Drop the amenities column
df = df.drop(['amenities'], axis=1)

# Display the updated df
df





Index(['index', 'id', 'latitude', 'longitude', 'host_since',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_listings_count', 'host_total_listings_count',
       'host_has_profile_pic', 'host_identity_verified',
       'calculated_host_listings_count', 'accommodates', 'bathrooms',
       'bedrooms', 'beds', 'amenities', 'has_availability', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'instant_bookable', 'minimum_nights', 'maximum_nights',
       'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'first_review', 'last_review', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'reviews_per_month',
       'room_Entire home/apt', 'room_Hotel room', 'room_Private room',
       'room_Shared room'],
      dt

,id,latitude,longitude,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,...,Air conditioning,Kitchen,Dedicated workspace,Heating,Hot water,Refrigerator,Free street parking,Self check-in,Shampoo,Washer
0,3917,40.744620,-73.904520,15832,1.701004,91.509003,78.592453,0,1.0,12.0,...,0,1,1,0,1,1,1,1,1,0
1,1885,40.753407,-73.934995,15563,1.000000,99.000000,23.000000,0,727.0,1336.0,...,1,1,1,1,1,1,0,0,0,0
2,1305,40.677090,-73.943810,15474,1.701004,91.509003,78.592453,0,1.0,1.0,...,1,1,0,1,0,0,0,0,0,0
3,19328,40.795760,-73.971570,18722,2.000000,70.000000,37.000000,0,36.0,79.0,...,1,1,1,1,1,0,0,0,0,1
4,16511,40.713590,-73.955400,16350,3.000000,100.000000,75.000000,0,1.0,1.0,...,0,1,0,1,1,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6722,7205,40.637960,-73.951360,16690,1.000000,100.000000,100.000000,0,2.0,2.0,...,0,1,1,1,1,1,1,1,1,0
6723,3954,40.823720,-73.945460,16877,4.000000,0.000000,33.000000,0,7.0,8.0,...,1,0,1,1,0,0,0,1,0,0
6724,1358,40.755094,-73.937260,15563,1.000000,99.000000,23.000000,0,727.0,1336.0,...,1,1,1,1,1,1,0,0,0,0
6725,2793,40.781580,-73.984780,15351,1.000000,100.000000,100.000000,1,1.0,3.0,...,1,1,1,0,1,1,0,0,1,0


## Turn

In [303]:
# Save the DataFrame as a CSV file
df.to_csv('test_processed.csv', index=False)